In [7]:
import pandas as pd
import numpy as np

In [8]:
CUBE = pd.read_excel("CUBE\Group 3-Cube.xlsx")
HOBO_low = pd.read_csv("HOBO\Group3-Low.csv", encoding="windows-1252")
HOBO_mid = pd.read_csv("HOBO\Group3-Mid.csv", encoding="windows-1252")
HOBO_top = pd.read_csv("HOBO\Group3-Top.csv", encoding="windows-1252")

In [9]:
T_mrt = CUBE["Mean Radiant Temperature (*C)"]

CUBE["Full DateTime"] = pd.to_datetime(
    pd.Timestamp.today().strftime("%Y-%m-%d") + " " + CUBE.iloc[:, 1].astype(str)
)
# Convert time column to datetime before extracting min/max
CUBE["Full DateTime"] = pd.to_datetime(CUBE["Full DateTime"])
start_time = CUBE["Full DateTime"].min()
end_time = CUBE["Full DateTime"].max()

# Generate full second-by-second time range
time_range = pd.date_range(start=start_time, end=end_time, freq='1S')
full_time_df = pd.DataFrame({"Full DateTime": time_range})

# Compute average temperature from HOBO sensors
T_avg = (HOBO_low.iloc[:, 1] + HOBO_mid.iloc[:, 1] + HOBO_top.iloc[:, 1]) / 3

# Extract Mean Radiant Temperature with Time
T_mrt = CUBE[["Full DateTime", "Mean Radiant Temperature (*C)"]]
T_mrt = pd.merge(full_time_df, T_mrt, how="left", on="Full DateTime")
T_mrt ["Mean Radiant Temperature (*C)"].fillna(0, inplace=True)

T_avg = T_avg.reset_index(drop=True)
T_mrt = T_mrt["Mean Radiant Temperature (*C)"].reset_index(drop=True)
HOBO_low = HOBO_low.reset_index(drop=True)

T_op = []
for i in range(T_avg.size):
    air_speed = HOBO_low["Air_speed (m/s)"][i]
    if T_mrt[i] == 0:
        continue
    if air_speed < 0.2:
        T_op.append((T_mrt[i] + T_avg[i]) / 2)
    elif air_speed < 0.6:
        T_op.append((0.4 * T_mrt[i] + 0.6 * T_avg[i]))
    else:
        T_op.append((0.3 * T_mrt[i] + 0.7 * T_avg[i]))
print(len(T_op))


5373


C:\Users\aarne\AppData\Local\Temp\ipykernel_22244\429395739.py:12: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  time_range = pd.date_range(start=start_time, end=end_time, freq='1S')
C:\Users\aarne\AppData\Local\Temp\ipykernel_22244\429395739.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  T_mrt ["Mean Radiant Temperature (*C)"].fillna(0, inplace=True)
